# parking

*This problem's setting is borrowed from: B. Nelson (2010). Stochastic Modeling: Analysis and Simulation. Dover, New York.*

Planning for construction of the the proposed Massive Mall &mdash; the largest shopping mall and indoor golf course in the world &mdash; includes determining the amount of customer parking to provide. Massive Mall will have a customer parking garage with 3 floors. The planners need to figure out how many parking spaces per floor the garage should have: if they are generous with the size of a parking space, they can fit 950 spaces in 1 floor; if they are stingy, then they can fit 960 spaces in 1 floor.

Through forecasts based on historical data, surveys, and malls in similar markets, the planners have data on customer traffic to the mall and the amount of time customers spend shopping. In particular, on Saturdays:

* the time between customer arrivals to the parking garage is forecast to be exponentially distributed with mean 1/1000 hours (i.e. 1000 customers per hour).

* the time a customer spends in the mall is forecast to be exponentially distributed with mean 3 hours.


The mall will be open from 9 am to 9 pm on Saturdays.


1. Simulate the arrival and departure of customers from the parking garage on Saturdays, assuming that each floor has 950 spaces. Using 50 replications, give a point and interval estimate for the mean time average number of customers waiting for a parking spot.

2. Simulate the arrival and departure of customers from the parking garage on Saturdays, assuming that each floor has 960. Using 50 replications, give a point and interval estimate for the mean time average number of customers waiting for a parking spot.

3. Is it worth being stingy with the size of a parking space? In particular, are the mean time average number of customers waiting for a parking spot statistically different when each floor has 950 spaces vs. 960 spaces? Use the Levene test for equal variances and the $t$-test for equal means to answer this question.

*Note.* These replicated simulations might take a few minutes (as opposed to a few seconds). 

We can approach this problem in a similar way to the Fantastic Dan problem by thinking of each parking space as a "server", and the time a customer spends in the mall as the "service time". Below are the parameters, processes and resources for a SimPy model that simulates this parking garage system. 

Note that 

* the number of parking spaces per floor, `P.nSpacesPerFloor`, is set to `None` &mdash; it will be set later.
* all the time quantities are in minutes - this will make the output easier to interpret.

In [1]:
# Import everything from SimPy
from SimPy.Simulation import *

# Import mean, std, sqrt functions from NumPy
from numpy import mean, std, sqrt

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, exponential

# Import t random variable from scipy.stats
from scipy.stats import t, levene, ttest_ind


##### Parameters #####
class P:
    ### To make the output easier to interpret, ###
    ### let's make all time quantities in minutes ###
    
    # Interarrival times are exponentially distributed with mean 1/1000 hours
    interarrivalTimeMean = 1/1000 * 60
    
    # Parking times are exponentially distributed with mean 3 hours
    parkingTimeMean = 3 * 60
    
    # Number of floors
    nFloors = 3
    
    # Number of parking spaces per floor
    nSpacesPerFloor = None
    
    # Simulation time: 12 hours
    simulationTimeMax = 12 * 60
    

##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue for parking garage
        yield request, self, R.garage
        
        # Customer enters parking garage and starts shopping
        parkingTime = exponential(scale = P.parkingTimeMean)
        yield hold, self, parkingTime
        
        # Customer finishes shopping, leaves garage
        yield release, self, R.garage

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

            
##### Resources #####
class R:
    # Parking garage
    garage = None

Below is the function `model()` that actually runs the simulation. Note that

* the function takes a value as input for the seed.
* the capacity of `R.garage` is `P.nFloors * P.nSpacesPerFloor`.
* the built-in monitors for `R.garage` has been activated.
* the function outputs the time average number of customers in the parking garage queue.

In [2]:
##### Model #####
def model(inputSeed):
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator
    seed(inputSeed)

    # Create the garage resource
    R.garage = Resource(capacity = P.nFloors * P.nSpacesPerFloor, monitored = True)

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

    # Compute time average number of customers in queue
    tAvgCustQ = R.garage.waitMon.timeAverage()
    
    # Return time average number of customers in queue
    return tAvgCustQ

#### System 1: 950 spaces per floor

In [3]:
# Number of floors
P.nSpacesPerFloor = 950

# Number of replications
n = 50

# Replicate simulation and capture performance measure
tAvgCustQ1Obs = [model(123*i) for i in range(n)]

## Observed sample mean
tAvgCustQ1SM = mean(tAvgCustQ1Obs)

# Observed sample standard deviation
tAvgCustQ1SSD = std(tAvgCustQ1Obs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
tAvgCustQ1CIL = tAvgCustQ1SM - t.ppf(1 - alpha/2, n - 1) * tAvgCustQ1SSD / sqrt(n)
tAvgCustQ1CIR = tAvgCustQ1SM + t.ppf(1 - alpha/2, n - 1) * tAvgCustQ1SSD / sqrt(n)

print("GARAGE WITH {0} SPACES PER FLOOR".format(P.nSpacesPerFloor))
print("Time average number of customers in queue:")
print("  Sample mean: {0}".format(tAvgCustQ1SM))
print("  Sample standard deviation: {0}".format(tAvgCustQ1SSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, tAvgCustQ1CIL, tAvgCustQ1CIR))

GARAGE WITH 950 SPACES PER FLOOR
Time average number of customers in queue:
  Sample mean: 20.805830062893804
  Sample standard deviation: 14.617321963295543
  95.0% confidence interval: [16.651633130614016, 24.960026995173592]


#### System 2: 960 spaces per floor

In [4]:
# Number of floors
P.nSpacesPerFloor = 960

# Number of replications
n = 50

# Replicate simulation and capture performance measure
tAvgCustQ2Obs = [model(123*i) for i in range(n)]

## Observed sample mean
tAvgCustQ2SM = mean(tAvgCustQ2Obs)

# Observed sample standard deviation
tAvgCustQ2SSD = std(tAvgCustQ2Obs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
tAvgCustQ2CIL = tAvgCustQ2SM - t.ppf(1 - alpha/2, n - 1) * tAvgCustQ2SSD / sqrt(n)
tAvgCustQ2CIR = tAvgCustQ2SM + t.ppf(1 - alpha/2, n - 1) * tAvgCustQ2SSD / sqrt(n)

print("GARAGE WITH {0} SPACES PER FLOOR".format(P.nSpacesPerFloor))
print("Time average number of customers in queue:")
print("  Sample mean: {0}".format(tAvgCustQ2SM))
print("  Sample standard deviation: {0}".format(tAvgCustQ2SSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, tAvgCustQ2CIL, tAvgCustQ2CIR))

GARAGE WITH 960 SPACES PER FLOOR
Time average number of customers in queue:
  Sample mean: 12.643059234130442
  Sample standard deviation: 11.980747452983216
  95.0% confidence interval: [9.23816848597723, 16.047949982283654]


#### Comparing System 1 and System 2

First, we perform the Levene test: are the variances of the time average number of customers in the queue for both systems equal?

In [5]:
(LeveneTS, LevenePValue) = levene(tAvgCustQ1Obs, tAvgCustQ2Obs)
print("Levene test p-value = {0}".format(LevenePValue))

Levene test p-value = 0.1310033862658699


Assuming a significance level of 0.05, we **do not reject** the null hypothesis of the Levene test. Therefore, we can assume that the variances in both systems are equal.

Next, we perform the $t$-test for equal means, assuming equal variances.

In [6]:
(tTS, tPValue) = ttest_ind(tAvgCustQ1Obs, tAvgCustQ2Obs, equal_var = True)
print("t-test p-value = {0}".format(tPValue))

t-test p-value = 0.0029081909929704348


Assuming a significance level of 0.05, we **reject** the null hypothesis of the $t$-test.
Therefore, we can assume that the mean time average number of customers in the queue are actually different across systems: the improvement in the mean time average number of customers in the queue (approximately -8) achieved by increasing the number of spaces per floor from 950 to 960 is statistically significant.

Of course, whether this difference of -8 is worth the increased number of customers complaining about cramped parking spaces and dinged cars depends on what the planners value.